# Agentic RecSys

We already did a variation of this in pointwise, pairwise and listwise recommendation

In [4]:
import pandas as pd

# Load datasets
customers = pd.read_csv('data/olist_customers_dataset.csv')
geolocations = pd.read_csv('data/olist_geolocation_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
payments = pd.read_csv('data/olist_order_payments_dataset.csv')
reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
product_category_translation = pd.read_csv('data/product_category_name_translation.csv')

In [5]:
# Recreate the merged orders_products_df
orders_customers_df = orders.merge(customers, on='customer_id')
orders_items_df = orders_customers_df.merge(order_items, on='order_id')
orders_reviews_df = orders_items_df.merge(reviews, on='order_id', how='left')
orders_products_df = orders_reviews_df.merge(products, on='product_id')


In [6]:
def pointwise_recommender(user_id, product_id):
    # Get user's review history
    user_reviews = orders_products_df[orders_products_df['customer_unique_id'] == user_id]['review_comment_message'].dropna().tolist()
    user_reviews_text = ' '.join(user_reviews)
    
    # Get the product information
    product_info = products_with_category_df[products_with_category_df['product_id'] == product_id].iloc[0]
    
    # Simulate LLM call
    gpt_caller = OpenAICaller("You are an assistant who helps with product recommendations based on user review history.")
    response = gpt_caller.call_llm(f"Here is the user's review history: {user_reviews_text}. Based on this history, will the user like the product '{product_info['product_category_name_english']}'? Please provide a rating between 1 and 5.")
    
    return response


In [7]:
def pairwise_recommender(user_id, product_id1, product_id2):
    # Get user's review history
    user_reviews = orders_products_df[orders_products_df['customer_unique_id'] == user_id]['review_comment_message'].dropna().tolist()
    user_reviews_text = ' '.join(user_reviews)
    
    # Get information for both products
    product_info1 = products_with_category_df[products_with_category_df['product_id'] == product_id1].iloc[0]
    product_info2 = products_with_category_df[products_with_category_df['product_id'] == product_id2].iloc[0]
    
    # Simulate LLM call
    gpt_caller = OpenAICaller("You are an assistant who helps with product recommendations based on user review history.")
    response = gpt_caller.call_llm(f"Here is the user's review history: {user_reviews_text}. Based on this history, which product would the user prefer: '{product_info1['product_category_name_english']}' or '{product_info2['product_category_name_english']}'? Please provide your preference and reasoning.")
    
    return response


In [8]:
def listwise_recommender(user_id, product_ids):
    # Get user's review history
    user_reviews = orders_products_df[orders_products_df['customer_unique_id'] == user_id]['review_comment_message'].dropna().tolist()
    user_reviews_text = ' '.join(user_reviews)
    
    # Get information for all products
    product_info_list = [products_with_category_df[products_with_category_df['product_id'] == product_id].iloc[0] for product_id in product_ids]
    product_names = ', '.join([product_info['product_category_name_english'] for product_info in product_info_list])
    
    # Simulate LLM call
    gpt_caller = OpenAICaller("You are an assistant who helps with product recommendations based on user review history.")
    response = gpt_caller.call_llm(f"Here is the user's review history: {user_reviews_text}. Based on this history, how would the user rank the following products: {product_names}? Please provide the ranking from most to least preferred.")
    
    return response
